In [1]:
import pandas as pd

# from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB, CategoricalNB
import re
import nltk
from nltk.corpus import stopwords

SEED = 3

In [2]:
df = pd.read_csv('../data/binary.csv')
df

,text,hatespeech_comb
0,@__andrea__b \r\nO cara vive em outro mundo\r\...,1.0
1,@_carmeloneto Estes incompetentes não cuidam n...,0.0
2,@_carmeloneto \r\nOs 'cumpanhero' quebraram to...,0.0
3,@_GlitteryKisses é isso não conseguem pensar n...,0.0
4,@_iglira bom dia macaco branco haha,1.0
...,...,...
25829,“odeio sulista” todos: simmm🥹😭🙏🏻😮‍💨🤮 “odeio no...,0.0
25830,👀🤗 é cada merda que eu vejo no Twitter não tem...,0.0
25831,👇👇👇👇esse cara é um FDP não sei como ainda não ...,1.0
25832,"🔥No Brasil que nós queremos, figuras como este...",1.0


In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\laura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
def preprocess_text(text):
    if not isinstance(text, str):
        text = str(text)
    # Remover IDs e padrões >>, mantendo expressões
    text = re.sub(r'>>\d+', '', text)
    # Remover menções como @usuário
    text = re.sub(r'@\w+', '', text)
    # Manter emojis, pontos de interrogação e exclamação
    text = re.sub(r'[^\w\s!?]', '', text)
    # Converter para minúsculas
    text = text.lower()
    return text

df['txt_clean'] = df['text'].apply(preprocess_text)
df.drop(columns=['text'], inplace=True)
display(df)

,hatespeech_comb,txt_clean
0,1.0,\r\no cara vive em outro mundo\r\nnão no mund...
1,0.0,estes incompetentes não cuidam nem do povo br...
2,0.0,\r\nos cumpanhero quebraram todas as regras
3,0.0,é isso não conseguem pensar no sentido lato p...
4,1.0,bom dia macaco branco haha
...,...,...
25829,0.0,odeio sulista todos simmm odeio nordestino tod...
25830,0.0,é cada merda que eu vejo no twitter não tem como
25831,1.0,esse cara é um fdp não sei como ainda não mand...
25832,1.0,no brasil que nós queremos figuras como este d...


### **Exploratory Data Analysis (EDA)**

In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25834 entries, 0 to 25833
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   hatespeech_comb  25834 non-null  float64
 1   txt_clean        25834 non-null  object 
dtypes: float64(1), object(1)
memory usage: 403.8+ KB
None


In [6]:
print(df['hatespeech_comb'].value_counts())

hatespeech_comb
0.0    15206
1.0    10628
Name: count, dtype: int64


In [7]:
df.describe()

,hatespeech_comb
count,25834.000000
mean,0.411396
std,0.492096
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [8]:
df['txt_clean']

0         \r\no cara vive em outro mundo\r\nnão no mund...
1         estes incompetentes não cuidam nem do povo br...
2              \r\nos cumpanhero quebraram todas as regras
3         é isso não conseguem pensar no sentido lato p...
4                               bom dia macaco branco haha
                               ...                        
25829    odeio sulista todos simmm odeio nordestino tod...
25830     é cada merda que eu vejo no twitter não tem como
25831    esse cara é um fdp não sei como ainda não mand...
25832    no brasil que nós queremos figuras como este d...
25833     vai poupar o nordestino de ouvir esse lixo de...
Name: txt_clean, Length: 25834, dtype: object

In [9]:
df_anger = df[df['hatespeech_comb'] == 1]
print(df_anger)

       hatespeech_comb                                          txt_clean
0                  1.0   \r\no cara vive em outro mundo\r\nnão no mund...
4                  1.0                         bom dia macaco branco haha
17                 1.0       perfume pra mão jovem amantepra mão cappice?
18                 1.0          estimo que se foda será mais profissional
19                 1.0                      bom dia sapatao da minha vida
...                ...                                                ...
25826              1.0             vc nordestino lítio urânio lixo humano
25827              1.0  nordestino só se faz de coitado disse a nojent...
25831              1.0  esse cara é um fdp não sei como ainda não mand...
25832              1.0  no brasil que nós queremos figuras como este d...
25833              1.0   vai poupar o nordestino de ouvir esse lixo de...

[10628 rows x 2 columns]


In [10]:
df_anger = df[df['hatespeech_comb'] == 0]
print(df_anger['txt_clean'])

1         estes incompetentes não cuidam nem do povo br...
2              \r\nos cumpanhero quebraram todas as regras
3         é isso não conseguem pensar no sentido lato p...
5         imagino  ow mandei fazer um body opressor pra...
6                      parabéns brou! menina é bom demais!
                               ...                        
25819    carreira na europa esse merda n se firmou em l...
25825    nordestinos desgraçados bolsonaristas espalham...
25828    nunca mais visito o nordeste povo mal agradeci...
25829    odeio sulista todos simmm odeio nordestino tod...
25830     é cada merda que eu vejo no twitter não tem como
Name: txt_clean, Length: 15206, dtype: object


In [13]:
print(df.iloc[0][0], f'Hate.speech: {df.iloc[0][1]}', sep='\n')
print('\n')
print(df.iloc[77][0], f'Hate.speech: {df.iloc[77][1]}', sep='\n')

1.0
Hate.speech:  
o cara vive em outro mundo
não no mundo real
refugiados são os que vivem
nas favelas vizinhas as suas fortalezas


0.0
Hate.speech: acho mal que não tenham metido a carreira da bárbara guimarães neste in memoriam
oscars


C:\Users\laura\AppData\Local\Temp\ipykernel_61976\1476165550.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df.iloc[0][0], f'Hate.speech: {df.iloc[0][1]}', sep='\n')
C:\Users\laura\AppData\Local\Temp\ipykernel_61976\1476165550.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df.iloc[77][0], f'Hate.speech: {df.iloc[77][1]}', sep='\n')


### **Model training**

#### **Preprocessing**

In [14]:
X = df['txt_clean']
y = df['hatespeech_comb']

In [15]:
print(f'X nulls: {X.isna().sum()}')
print(f'y nulls: {y.isna().sum()}')

X nulls: 0
y nulls: 0


In [16]:
def get_pipe(model, params: dict):
    return Pipeline(steps=[
        ('vectorizer', TfidfVectorizer()),
        ('model', model(**params)),
    ])

def validate_pipe(pipe, X, y):
    accuracy = cross_val_score(pipe, X, y, cv=StratifiedKFold(n_splits=5), scoring='accuracy')
    precision = cross_val_score(pipe, X, y, cv=StratifiedKFold(n_splits=5), scoring='precision')
    recall = cross_val_score(pipe, X, y, cv=StratifiedKFold(n_splits=5), scoring='recall')
    f1 = cross_val_score(pipe, X, y, cv=StratifiedKFold(n_splits=5), scoring='f1')

    result_matrix = pd.DataFrame.from_dict({
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    })

    print(
        f'Accuracy: {result_matrix['accuracy'].mean():.1%}',
        f'Precision: {result_matrix['precision'].mean():.1%}',
        f'Recall: {result_matrix['recall'].mean():.1%}',
        f'F1: {result_matrix['f1'].mean():.1%}',
        sep='\n',
    )

    return result_matrix

In [17]:
multinomialnb_pipe = get_pipe(MultinomialNB, {})
validate_pipe(multinomialnb_pipe, X, y)

Accuracy: 71.2%
Precision: 62.4%
Recall: 74.8%
F1: 66.5%


,accuracy,precision,recall,f1
0,0.567834,0.459985,0.289276,0.355183
1,0.712793,0.609184,0.842427,0.707067
2,0.893942,0.848191,0.904045,0.875228
3,0.727114,0.656182,0.706824,0.680562
4,0.658924,0.546839,0.997176,0.706333


In [18]:
complementnb_pipe = get_pipe(ComplementNB, {})
validate_pipe(complementnb_pipe, X, y)

Accuracy: 65.0%
Precision: 54.9%
Recall: 83.9%
F1: 65.7%


,accuracy,precision,recall,f1
0,0.535514,0.438288,0.457667,0.447768
1,0.628218,0.527891,0.912512,0.668850
2,0.817689,0.702879,0.964722,0.813243
3,0.681246,0.575110,0.861176,0.689655
4,0.588850,0.500118,0.999059,0.666562


In [19]:
bernoullinb_pipe = get_pipe(BernoulliNB, {})
validate_pipe(bernoullinb_pipe, X, y)

Accuracy: 72.0%
Precision: 82.7%
Recall: 42.1%
F1: 48.2%


,accuracy,precision,recall,f1
0,0.593768,1.000000,0.012700,0.025081
1,0.871686,0.949048,0.727187,0.823435
2,0.900523,0.917184,0.833490,0.873337
3,0.627443,0.741546,0.144471,0.241827
4,0.604723,0.526620,0.386353,0.445711
